In [1]:
import pandas as pd
import ast

In [2]:
df = pd.read_csv('data/raw_data.csv')
df.head(1)

,jobId,jobTitle,jobUrl,userId,companyLogo,alias,createdOn,approvedOn,expiredOn,lastUpdatedOn,...,isMobileHotJob,isMobileTopJob,isBoldAndRedJob,isUrgentJob,isUrgentJobM,onlineOn,visibilityDisplay,priorityOrder,nearestGeoLoc,profilePublishedSiteMask
0,1680760,Nhân Viên Kinh Doanh MT - GT,NaN,7323468,NaN,nhan-vien-kinh-doanh-mt-gt-2,NaN,2023-08-11T11:22:45+07:00,2023-09-10T23:59:59+07:00,NaN,...,True,True,False,False,False,2023-08-11T11:22:45+07:00,False,2023-08-11T11:22:45,NaN,0


<h5>Loại bỏ các cột không cần thiết

In [3]:
list_drop = ['jobUrl','userId','companyLogo','alias', 'createdOn', 'approvedOn', 'expiredOn', 'lastUpdatedOn', 'lastSyncedOn', 
       'companyId', 'companyProfile', 'companySizeId',
       'contactName', 'createdBy', 'isOnline', 'isActive', 'isApproved',
       'isUnpaidDisable', 'isCompleted', 'isShowLogo', 'isShowLogoInSearch','languageSelectedId', 'redirectTo', 'requiredCoverLetter',
       'languageId', 'locations', 'canonical',
       'isShowContact', 'salaryRangeId', 'requiredResume',
       'companySizeVI', 'languageSelectedVI','simpleServices', 'services',
       'emailAddress', 'isMobileHotJob', 'isMobileTopJob',
       'isBoldAndRedJob', 'isUrgentJob', 'isUrgentJobM', 'onlineOn',
       'visibilityDisplay', 'priorityOrder', 'nearestGeoLoc','profilePublishedSiteMask']
df = df.drop(list_drop, axis=1)

<h5>Loại bỏ các cột có giá trị NaN và 0

In [4]:
df = df.dropna(axis=1, how='all')
df.columns.size

24

In [5]:
df = df.loc[:,(df != 0).any(axis = 0)]
df.columns.size

24

<h5>Loại bỏ giá trị trùng lặp

In [6]:
df = df.drop_duplicates(keep = 'first')
df.shape

(8978, 24)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8978 entries, 0 to 8999
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   jobId              8978 non-null   int64 
 1   jobTitle           8978 non-null   object
 2   companyName        8978 non-null   object
 3   isAnonymous        8978 non-null   bool  
 4   isSalaryVisible    8978 non-null   bool  
 5   jobDescription     8976 non-null   object
 6   jobRequirement     8978 non-null   object
 7   jobLevelId         8978 non-null   int64 
 8   salary             8978 non-null   int64 
 9   salaryMax          8978 non-null   int64 
 10  salaryMin          8978 non-null   int64 
 11  typeWorkingId      8978 non-null   int64 
 12  skills             8974 non-null   object
 13  benefits           8975 non-null   object
 14  workingLocations   8974 non-null   object
 15  address            8786 non-null   object
 16  industries         8974 non-null   object
 17  

<h5> Xử lý cột 'companySize'

In [8]:
df['companySize'].value_counts()

companySize
100-499          2256
25-99            1793
1,000-4,999      1207
500-999           763
10-24             667
5,000-9,999       329
10,000-19,999     215
Less Than 10      108
20,000-49,999      43
Over 50,000        42
Name: count, dtype: int64

In [9]:
def clean_companySize(s): 
    if s == "100-499": return (100, 499)
    elif s == "25-99": return (25, 99)
    elif s == "1,000-4,999": return (1000, 4999)
    elif s == "500-999": return (500, 999)
    elif s == "10-24": return (10, 24)
    elif s == "5,000-9,999": return (5000, 9999)
    elif s == "10,000-19,999": return (10000, 19999)
    elif s == "Less Than 10": return (0, 10)
    elif s == "20,000-49,999": return (20000, 49999)
    elif s == "Over 50,000": return (50000, 99999)
    else: return None
#thay thế vào giá trị trong cột cũ
temp_df = df.copy()
temp_df['companySize'] = df['companySize'].apply(lambda s: clean_companySize(s))
temp_df['companySize'].value_counts().sum()

7423

<h5>Xử lý cột 'workingLocations'

In [10]:
temp_df["workingLocations"].value_counts()

workingLocations
[{'workingLocationId': 119997, 'addressId': 774, 'cityId': 24, 'districtId': 0, 'address': 'Hà Nội, Vietnam', 'geoLoc': {'lat': 21.027764, 'lon': 105.83416}, 'cityName': 'Ha Noi', 'cityNameVI': 'Hà Nội'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                 42
[{'workingLocationId': 17, 'addressId': 14572, 'cityId': 24, 'districtId': 10, 'address': 'Hà Nội, Việt Nam', 'geoLoc': {'lat': 21.027764, 'lon': 105.83416}, 'cityName': 'Ha Noi', 'cityNameVI': 'Hà Nội'}]                                                                                                                           

In [11]:
#loại bỏ nan
temp_df = temp_df.dropna(subset=['workingLocations'])
#chuyển đổi chuỗi json
temp_df['workingLocations'] = temp_df['workingLocations'].apply(ast.literal_eval)
def clean_workingLocations(x):
    cityName = {} 
    for i in x: 
        #lấy Id thành phố và tên
        cityName[i.get("cityId")] = i.get("cityNameVI")
    return cityName
temp_df['workingLocations'] = temp_df['workingLocations'].apply(lambda x: clean_workingLocations(x))
temp_df['workingLocations']

0         {24: 'Hà Nội'}
1         {24: 'Hà Nội'}
2         {24: 'Hà Nội'}
3         {24: 'Hà Nội'}
4         {24: 'Hà Nội'}
              ...       
8995      {24: 'Hà Nội'}
8996      {24: 'Hà Nội'}
8997    {37: 'Lạng Sơn'}
8998      {24: 'Hà Nội'}
8999      {24: 'Hà Nội'}
Name: workingLocations, Length: 8974, dtype: object

<h5>Tương tự cho 3 cột 'skills', 'benefits','industries'

In [12]:
temp_df = temp_df.dropna(subset=['skills'])
temp_df['skills'] = temp_df['skills'].apply(ast.literal_eval)
temp_df['benefits'] = temp_df['benefits'].apply(ast.literal_eval)
temp_df['industries'] = temp_df['industries'].apply(ast.literal_eval)

In [13]:
def clean_skills(x):
    skills= {} 
    for i in x: 
        skills[i.get("skillId")] = i.get("skillName")
    return skills

def clean_benefits(x):
    benefits= {} 
    for i in x: 
        benefits[i.get("benefitId")] = i.get("benefitName")
    return benefits
def clean_industries(x): 
    industries= {} 
    for i in x: 
        industries[i.get("industryId")] = i.get("industryName")
    return industries

In [14]:
temp_df['skills'] = temp_df['skills'].apply(lambda x: clean_skills(x))
temp_df['benefits'] = temp_df['benefits'].apply(lambda x: clean_benefits(x))
temp_df['industries'] = temp_df['industries'].apply(lambda x: clean_industries(x))
temp_df[['skills', 'benefits','industries']]

,skills,benefits,industries
0,"{1919: 'Kênh MT', 1921: 'Hàng Tiêu Dùng', 666:...","{1: 'Bonus', 2: 'Healthcare Plan', 10: 'Team A...","{11: 'Customer Service', 32: 'Retail/Wholesale..."
1,"{27810: 'Marketing Materials Strategy', 333851...","{1: 'Bonus', 2: 'Healthcare Plan', 4: 'Training'}","{3: 'Advertising/Promotion/PR', 27: 'Marketing..."
2,"{93: 'Kế Toán Tổng Hợp', 941661: 'Hạch Toán Ch...","{1: 'Bonus', 2: 'Healthcare Plan', 10: 'Team A...","{1: 'Accounting', 58: 'Auditing', 59: 'Finance..."
3,"{100: 'English', 651: 'Trade Marketing', 15900...","{1: 'Bonus', 2: 'Healthcare Plan', 9: 'Travel ...","{3: 'Advertising/Promotion/PR', 27: 'Marketing..."
4,"{17750: 'Tìm Kiếm Khách Hàng', 354: 'Chăm Sóc ...","{1: 'Bonus', 4: 'Training', 5: 'Awards'}","{33: 'Sales', 42: 'Banking', 59: 'Finance/Inve..."
...,...,...,...
8995,"{618943: 'Government Organizations', 236965: '...","{1: 'Bonus', 5: 'Awards', 15: 'Others'}","{8: 'Consulting', 58: 'Auditing', 69: 'Plannin..."
8996,"{618943: 'Government Organizations', 941031: '...","{1: 'Bonus', 5: 'Awards', 15: 'Others'}","{8: 'Consulting', 58: 'Auditing', 69: 'Plannin..."
8997,"{67: 'Xuất Nhập Khẩu', 26367: 'Booking', 23667...","{2: 'Healthcare Plan', 3: 'Paid Leave', 10: 'T...","{19: 'Export-Import', 36: 'Freight/Logistics'}"
8998,"{9428: 'Chinese', 346516: 'Business Deveplomen...","{1: 'Bonus', 5: 'Awards', 15: 'Others'}","{8: 'Consulting', 11: 'Customer Service', 58: ..."


<h5>Chuyển đổi cột typeWorkingId

In [15]:
temp_df["typeWorkingId"].value_counts().sort_index().index

Index([1, 2, 3, 4, 5, 6, 7], dtype='int64', name='typeWorkingId')

In [16]:
def convert_typeWorkingName(id):
    if (id == 1):
        return 'Full-time'
    elif (id == 2):
        return 'Part-time'
    elif (id == 3):
        return 'Internship'
    elif (id == 4):
        return 'Online jobs'
    elif (id == 5):
        return 'Freelancer'
    elif (id == 6):
        return 'Seasonal'
    else:
        return 'Other'

In [17]:
#tạo thêm cột workingName với workingID tương ứng
temp_df['typeWorkingName'] = [convert_typeWorkingName(x) for x in temp_df['typeWorkingId']]

<h5>Tìm các cột có giá trị thiếu

In [18]:
missing_values = temp_df.isnull().sum()
columns_with_missing_values = missing_values[missing_values > 0]
print(columns_with_missing_values)

jobDescription       2
address            192
companySize       1554
dtype: int64


<h5><i>Nhận thấy cột 'jobDescription' và 'address' không quan trọng nên ta bổ sung giá trị cho cột 'companySize' bằng giá trị mode

In [19]:
mode_value = temp_df['companySize'].mode().values[0]
temp_df['companySize'].fillna(temp_df['companySize'].apply(lambda x: mode_value if pd.isnull(x) else x), inplace = True)

In [20]:
temp_df.to_csv('data/new_data.csv',index = False)